In [1]:
import torch
torch.cuda.is_available()

True

In [2]:
!pip install datasets transformers huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 14.8 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [3]:
!apt-get install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [7]:
from datasets import load_dataset
imdb = load_dataset("imdb")

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [8]:
small_train_dataset = imdb["train"].shuffle(seed=42).select([i for i in list(range(3000))])
small_test_dataset = imdb["test"].shuffle(seed=42).select([i for i in list(range(300))])


In [9]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [10]:
def preprocess_function(examples):
   return tokenizer(examples["text"], truncation=True)

tokenized_train = small_train_dataset.map(preprocess_function, batched=True)
tokenized_test = small_test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [11]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [12]:
from transformers import pipeline
classifier = pipeline("text-classification",model='bhadresh-savani/distilbert-base-uncased-emotion', return_all_scores=True)

config.json:   0%|          | 0.00/768 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [13]:
import pandas as pd

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Read the CSV file
data = pd.read_csv('/content/drive/MyDrive/522Project/combined_transcripts_Eng_to_Spa.csv')
data.columns = ['English', 'Spanish']

In [15]:
def get_predictions(texts):
    results = []
    for text in texts:
        if isinstance(text, str):
            # Make prediction and retrieve scores for each label
            prediction = classifier(text, truncation=True, max_length=512)
            # Extract scores from the prediction into a dictionary
            score_dict = {pred['label']: pred['score'] for pred in prediction[0]}
            results.append(score_dict)
        else:
            # Handle non-string or NaN inputs
            results.append({"sadness": None, "joy": None, "love": None, "anger": None, "fear": None, "surprise": None})
    return results


In [16]:
# Set display options to show full content in columns
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

In [ ]:
# Apply the function to the 'English' column
predictions_eng = get_predictions(data['English'])

# Convert predictions to DataFrame
predictions_df_eng = pd.DataFrame(predictions_eng)


In [ ]:
# Apply the function to the 'Spanish' column
predictions_spa = get_predictions(data['Spanish'])

# Convert predictions to DataFrame
predictions_df_spa = pd.DataFrame(predictions_spa)

In [ ]:
# Concatenate the original data with the predictions
final_data = pd.concat([data, predictions_df_eng, predictions_df_spa], axis=1)

In [ ]:
# The original columns from the dataset plus the English sentiment columns
original_plus_eng = final_data.columns.tolist()[:2] + [f'eng_{label}' for label in final_data.columns[2:8]]

# The Spanish sentiment columns with the 'spa_' prefix
spa_with_prefix = [f'spa_{label}' for label in final_data.columns[8:14]]

# Combine the two lists to get the full set of new column names
new_column_names = original_plus_eng + spa_with_prefix

# Assign the new column names to the DataFrame
final_data.columns = new_column_names

In [ ]:
# Now, the final_data should have the columns renamed as expected
final_data.head()

,English,Spanish,eng_sadness,eng_joy,eng_love,eng_anger,eng_fear,eng_surprise,spa_sadness,spa_joy,spa_love,spa_anger,spa_fear,spa_surprise
0,"It’s 2009 Alyssa Elliot is sitting at her desk at work, absolutely drowning in anxiety. The mortgage company has been calling and calling the debts and bills are piling up. And this isn’t like Alyssa, who has always done everything by the book. But that doesn’t matter to the debt collectors, the courts, even the people she’s confiding in, someone has to be held responsible. Even if that person only had the best intentions.","Es 2009, Alyssa Elliot está sentada en su escritorio en el trabajo, absolutamente ahogada en ansiedad. La compañía hipotecaria ha estado llamando y llamando porque las deudas y las facturas se están acumulando. Y esto no es propio de Alyssa, que siempre ha hecho todo según las reglas. Pero eso no les importa a los cobradores de deudas, a los tribunales, ni siquiera a las personas en las que ella confía: alguien tiene que ser considerado responsable. Incluso si esa persona sólo tuviera las mejores intenciones.",0.028962,0.015933,0.001228,0.918944,0.034355,0.000578,0.034715,0.587366,0.006106,0.294880,0.073891,0.003041
1,"You say things like I couldn’t have foreseen that this would happen, but like I really couldn’t. At the time I was in my early 20s I was young. I didn’t think that this would be what happens to me.","Dices cosas como si no hubiera podido prever que esto sucedería, pero realmente no podía. Cuando tenía poco más de 20 años, era joven. No pensé que esto sería lo que me pasaría.",0.048499,0.655383,0.002718,0.134418,0.155827,0.003155,0.033154,0.466453,0.013687,0.459348,0.023905,0.003452
2,"This is Last Day, a show about the moments that change us. I’m your host Stephanie Wittels Wachs. Today, the story of a decision that became $100,000 mistake, and how to make peace with that once you’re finally able to come up for air.","Este es Last Day, un programa sobre los momentos que nos cambian. Soy su anfitriona Stephanie Wittels Wachs. Hoy, la historia de una decisión que se convirtió en un error de 100.000 dólares y cómo hacer las paces con eso una vez que finalmente puedas salir a tomar aire.",0.002284,0.995309,0.000517,0.001167,0.000398,0.000326,0.031101,0.752294,0.005180,0.181601,0.027118,0.002706
3,"Alyssa grew up in Boise, Idaho with her mom, dad and sister and for all of you non Midwesterners,","Alyssa creció en Boise, Idaho con su mamá, su papá y su hermana y para todos ustedes que no son del Medio Oeste,",0.126080,0.331886,0.009107,0.416006,0.112349,0.004571,0.040082,0.553136,0.069975,0.304728,0.029099,0.002979
4,That is with an S not a Z biggest pet peeve.,"Eso es con una S, no con una Z, lo que más le molesta.",0.016062,0.569976,0.005678,0.379225,0.027054,0.002005,0.027138,0.385050,0.013899,0.530640,0.038923,0.004351


In [ ]:
# Save the dataframe with predictions to a new CSV
final_data.to_csv('/content/drive/MyDrive/522Project/SentAnalysis_Eng_to_Spa.csv', index=False)

In [ ]:
# Read the CSV file
data2 = pd.read_csv('/content/drive/MyDrive/522Project/combined_transcripts_Spa_to_Eng.csv')
data2.columns = ['Spanish', 'English']

In [ ]:
# Apply the function to the 'English' column
predictions_eng2 = get_predictions(data2['English'])

# Convert predictions to DataFrame
predictions_df_eng2 = pd.DataFrame(predictions_eng2)


In [ ]:
# Apply the function to the 'Spanish' column
predictions_spa2 = get_predictions(data2['Spanish'])

# Convert predictions to DataFrame
predictions_df_spa2 = pd.DataFrame(predictions_spa2)

In [ ]:
final_data2 = pd.concat([data2, predictions_df_eng2, predictions_df_spa2], axis=1)

In [ ]:
# The original columns from the dataset plus the English sentiment columns
original_plus_eng2 = final_data2.columns.tolist()[:2] + [f'eng_{label}' for label in final_data2.columns[2:8]]

# The Spanish sentiment columns with the 'spa_' prefix
spa_with_prefix2 = [f'spa_{label}' for label in final_data2.columns[8:14]]

# Combine the two lists to get the full set of new column names
new_column_names2 = original_plus_eng2 + spa_with_prefix2

# Assign the new column names to the DataFrame
final_data2.columns = new_column_names2

In [ ]:
#final_data2.head()

In [ ]:
# Save the dataframe with predictions to a new CSV
final_data2.to_csv('/content/drive/MyDrive/522Project/SentAnalysis_Spa_to_Eng.csv', index=False)

**Sentiment Analysis in idioms**

In [ ]:
#Adding all the data sets together for check sentiment analysis in idioms

idiom1 = pd.read_csv('/content/drive/MyDrive/522Project/idioms/Spanish_idioms.csv')
idiom2 = pd.read_csv('/content/drive/MyDrive/522Project/idioms/Spanish_idioms_1.csv')
idiom3 = pd.read_csv('/content/drive/MyDrive/522Project/idioms/Spanish_idioms_2.csv')
idiom4 = pd.read_csv('/content/drive/MyDrive/522Project/idioms/Spanish_idioms_3.csv')

idiom = pd.concat([idiom1, idiom2, idiom3, idiom4]).reset_index(drop=True)
idiom.drop(columns=['literal_translation', 'Example'], axis=1, inplace=True)
idiom.head()

,Idiom,Meaning
0,"A buen hambre, no hay pan duro.",Beggars can’t be choosers.
1,estar como unasopa,to be soaked to the bone
2,estar como un fideo,to be thin as a rail
3,estar como un queso,to be hot (sexually attractive)
4,no importar un pepino,to not care at all


In [ ]:
print(idiom.shape)

(2212, 2)


In [ ]:
predictions_eng3 = get_predictions(idiom['Meaning'])
predictions_df_eng3 = pd.DataFrame(predictions_eng3)

predictions_spa3 = get_predictions(idiom['Idiom'])
predictions_df_spa3 = pd.DataFrame(predictions_spa3)

In [ ]:
print(predictions_df_eng3.shape)
print(predictions_df_spa3.shape)

(2212, 6)
(2212, 6)


In [ ]:
final_data3 = pd.concat([idiom, predictions_df_eng3, predictions_df_spa3], axis=1)

# The original columns from the dataset plus the English sentiment columns
original_plus_eng3 = final_data3.columns.tolist()[:2] + [f'eng_{label}' for label in final_data3.columns[2:8]]

# The Spanish sentiment columns with the 'spa_' prefix
spa_with_prefix3 = [f'spa_{label}' for label in final_data3.columns[8:14]]

# Combine the two lists to get the full set of new column names
new_column_names3 = original_plus_eng3 + spa_with_prefix3

# Assign the new column names to the DataFrame
final_data3.columns = new_column_names3

# Save the dataframe with predictions to a new CSV
final_data3.to_csv('/content/drive/MyDrive/522Project/SentAnalysis_idioms.csv', index=False)

In [ ]:
final_data3.head()

,Idiom,Meaning,eng_sadness,eng_joy,eng_love,eng_anger,eng_fear,eng_surprise,spa_sadness,spa_joy,spa_love,spa_anger,spa_fear,spa_surprise
0,"A buen hambre, no hay pan duro.",Beggars can’t be choosers.,0.011603,0.002736,0.000632,0.980289,0.004451,0.000288,0.021514,0.119736,0.005086,0.836498,0.015410,0.001756
1,estar como unasopa,to be soaked to the bone,0.119516,0.031524,0.025592,0.770668,0.046732,0.005969,0.035885,0.668664,0.013412,0.225836,0.049863,0.006340
2,estar como un fideo,to be thin as a rail,0.090105,0.009540,0.002397,0.869479,0.027009,0.001470,0.013020,0.893818,0.006894,0.062987,0.020650,0.002632
3,estar como un queso,to be hot (sexually attractive),0.001021,0.003644,0.989614,0.001817,0.001617,0.002287,0.030938,0.639629,0.014984,0.248228,0.059439,0.006782
4,no importar un pepino,to not care at all,0.074329,0.271634,0.037179,0.604694,0.010450,0.001714,0.039221,0.289112,0.012358,0.633659,0.020962,0.004687


**USING MODEL GPT-4 FOR TRANSLATION**

Original Spanish transcripts translate to English with model gpt4

In [17]:
gpt1_spa_to_eng = pd.read_csv('/content/drive/MyDrive/522Project/gpt4/Amponas_gpt.csv', encoding='ISO-8859-1')
gpt2_spa_to_eng = pd.read_csv('/content/drive/MyDrive/522Project/gpt4/Barberias_gpt.csv')
gpt3_spa_to_eng = pd.read_csv('/content/drive/MyDrive/522Project/gpt4/Damitas_gpt.csv')
gpt4_spa_to_eng = pd.read_csv('/content/drive/MyDrive/522Project/gpt4/Farid_gpt.csv')
gpt5_spa_to_eng = pd.read_csv('/content/drive/MyDrive/522Project/gpt4/Fobias_gpt.csv')
gpt6_spa_to_eng = pd.read_csv('/content/drive/MyDrive/522Project/gpt4/Ovnis_gpt.csv')
gpt7_spa_to_eng = pd.read_csv('/content/drive/MyDrive/522Project/gpt4/Subio_gpt.csv')
gpt8_spa_to_eng = pd.read_csv('/content/drive/MyDrive/522Project/gpt4/Techos_gpt.csv')
gpt9_spa_to_eng = pd.read_csv('/content/drive/MyDrive/522Project/gpt4/Triangulo_gpt.csv')


gpt_spa_to_eng = pd.concat([gpt1_spa_to_eng, gpt2_spa_to_eng, gpt3_spa_to_eng, gpt4_spa_to_eng, gpt5_spa_to_eng, gpt6_spa_to_eng, gpt7_spa_to_eng, gpt8_spa_to_eng, gpt9_spa_to_eng]).reset_index(drop=True)
gpt_spa_to_eng.head()

,Original,gpt4
0,"¿Cómo estás, soy turbulent? Y yo la burrita burrona. Y estamos muy contentos que después del presupuesto para después de tal no, Claro que no. ¿Y usted a lo mejor va a decir Este por qué están solas? Pues yo no. Las temporadas, las temporadas son largas.","How are you, am I turbulent? And I the little stubborn donkey. And we are very happy that after the budget for after such no, of course not. And you might say why are they alone? Well, I don't. The seasons, the seasons are long."
1,"Son largas y aparte esta esta mención es. Para darle realce a nuestra gira, Claro, Claro, teníamos aquí a alguien sentado, no le íbamos a dar el. Lugar, pues no, porque no le íbamos a dar el lugar. En toda la persona, pero. Estamos también súper contentos porque a lo largo de este poquito tiempo que tenemos con el podcast del momento, pues han pasado cosas irreales.","The seasons are long and besides this mention is. To enhance our tour, of course, of course, we had someone sitting here, we wouldn't give him. Place, well no, because we wouldn't give him the place. In the whole person, but. We are also super happy because over this little time we have with the podcast of the moment, unreal things have happened."
2,"Y han pasado personas muy importantes por estas. No y las que se vienen o sea usted a lo mejor ahorita dijo, Ah, pues nos están diciendo que se termina, se termina esta temporada este porque no le dimos como un cierre porque usted vio que se cambió la escenografía. Ajá. ¿Y usted no lo vio, pero no le no habíamos tenido el tiempo de dar como un cierre formal?","And very important people have passed through these. Not and those to come or you might have just said, Ah, well they are telling us that it ends, this season ends because we didn't give it like a closure because you saw that the scenery changed. Uh-huh. And you didn't see it, but we hadn't had the time to give it a formal closure?"
3,"Ciclo formal y a partir de aquí el podcast del momento viene con un giro. Viene con pura celebridad, celebridad internacional, no ha pasado ninguna de las que ya grabamos. No, no ha pasado el. Lunes este domingo esto ya es el domingo, no, hombre, no van a soportar si de por sí no nos pueden ver.","A formal cycle and from here the podcast of the moment comes with a twist. It comes with pure celebrity, international celebrity, none of the ones we've already recorded have happened. No, it hasn't happened. This Monday this Sunday this is already Sunday, no, man, they're not going to stand it if they can't see us."
4,"Pero hay que aclarar que obviamente seguiremos con nuestro marketing, que son muchas figuras regias, tenemos ya ya tenemos pactados a varios que ustedes van a disfrutar en un futuro y conocerán un poquito más de la cultura regiomontana. Y también nos gustaría que en este momento y en este video.","But we must clarify that obviously we will continue with our marketing, which involves many regal figures, we already have several agreements that you will enjoy in the future and will learn a little more about Regiomontano culture. And we would also like you to at this moment and in this video."


In [18]:
predictions_eng4 = get_predictions(gpt_spa_to_eng['gpt4'])
predictions_df_eng4 = pd.DataFrame(predictions_eng4)

predictions_spa4 = get_predictions(gpt_spa_to_eng['Original'])
predictions_df_spa4 = pd.DataFrame(predictions_spa4)

In [19]:
final_data4 = pd.concat([gpt_spa_to_eng, predictions_df_eng4, predictions_df_spa4], axis=1)

# The original columns from the dataset plus the English sentiment columns
original_plus_eng4 = final_data4.columns.tolist()[:2] + [f'eng_{label}' for label in final_data4.columns[2:8]]

# The Spanish sentiment columns with the 'spa_' prefix
spa_with_prefix4 = [f'spa_{label}' for label in final_data4.columns[8:14]]

# Combine the two lists to get the full set of new column names
new_column_names4 = original_plus_eng4 + spa_with_prefix4

# Assign the new column names to the DataFrame
final_data4.columns = new_column_names4

# Save the dataframe with predictions to a new CSV
final_data4.to_csv('/content/drive/MyDrive/522Project/gpt_SentAnalysis_spa_to_eng.csv', index=False)

In [20]:
final_data4.head()

,Original,gpt4,eng_sadness,eng_joy,eng_love,eng_anger,eng_fear,eng_surprise,spa_sadness,spa_joy,spa_love,spa_anger,spa_fear,spa_surprise
0,"¿Cómo estás, soy turbulent? Y yo la burrita burrona. Y estamos muy contentos que después del presupuesto para después de tal no, Claro que no. ¿Y usted a lo mejor va a decir Este por qué están solas? Pues yo no. Las temporadas, las temporadas son largas.","How are you, am I turbulent? And I the little stubborn donkey. And we are very happy that after the budget for after such no, of course not. And you might say why are they alone? Well, I don't. The seasons, the seasons are long.",0.004452,0.001799,0.000400,0.992304,0.000880,0.000165,0.015954,0.027759,0.002183,0.938403,0.014955,0.000747
1,"Son largas y aparte esta esta mención es. Para darle realce a nuestra gira, Claro, Claro, teníamos aquí a alguien sentado, no le íbamos a dar el. Lugar, pues no, porque no le íbamos a dar el lugar. En toda la persona, pero. Estamos también súper contentos porque a lo largo de este poquito tiempo que tenemos con el podcast del momento, pues han pasado cosas irreales.","The seasons are long and besides this mention is. To enhance our tour, of course, of course, we had someone sitting here, we wouldn't give him. Place, well no, because we wouldn't give him the place. In the whole person, but. We are also super happy because over this little time we have with the podcast of the moment, unreal things have happened.",0.000344,0.998451,0.000293,0.000311,0.000280,0.000321,0.007161,0.954246,0.003007,0.027997,0.006280,0.001310
2,"Y han pasado personas muy importantes por estas. No y las que se vienen o sea usted a lo mejor ahorita dijo, Ah, pues nos están diciendo que se termina, se termina esta temporada este porque no le dimos como un cierre porque usted vio que se cambió la escenografía. Ajá. ¿Y usted no lo vio, pero no le no habíamos tenido el tiempo de dar como un cierre formal?","And very important people have passed through these. Not and those to come or you might have just said, Ah, well they are telling us that it ends, this season ends because we didn't give it like a closure because you saw that the scenery changed. Uh-huh. And you didn't see it, but we hadn't had the time to give it a formal closure?",0.025169,0.925640,0.002035,0.042370,0.003410,0.001376,0.019489,0.543833,0.009557,0.395938,0.027925,0.003257
3,"Ciclo formal y a partir de aquí el podcast del momento viene con un giro. Viene con pura celebridad, celebridad internacional, no ha pasado ninguna de las que ya grabamos. No, no ha pasado el. Lunes este domingo esto ya es el domingo, no, hombre, no van a soportar si de por sí no nos pueden ver.","A formal cycle and from here the podcast of the moment comes with a twist. It comes with pure celebrity, international celebrity, none of the ones we've already recorded have happened. No, it hasn't happened. This Monday this Sunday this is already Sunday, no, man, they're not going to stand it if they can't see us.",0.020715,0.400296,0.002824,0.559954,0.014306,0.001905,0.016694,0.508439,0.005596,0.436137,0.030408,0.002725
4,"Pero hay que aclarar que obviamente seguiremos con nuestro marketing, que son muchas figuras regias, tenemos ya ya tenemos pactados a varios que ustedes van a disfrutar en un futuro y conocerán un poquito más de la cultura regiomontana. Y también nos gustaría que en este momento y en este video.","But we must clarify that obviously we will continue with our marketing, which involves many regal figures, we already have several agreements that you will enjoy in the future and will learn a little more about Regiomontano culture. And we would also like you to at this moment and in this video.",0.000295,0.997686,0.000753,0.000564,0.000207,0.000497,0.011552,0.802784,0.008076,0.159256,0.015909,0.002423


Original English transcripts translate to Spanish with model gpt4

In [21]:
gpt1_eng_to_spa = pd.read_csv('/content/drive/MyDrive/522Project/gpt4/Flaps_gpt.csv', encoding='ISO-8859-1')
gpt2_eng_to_spa = pd.read_csv('/content/drive/MyDrive/522Project/gpt4/Highway_gpt.csv', encoding='ISO-8859-1')
gpt3_eng_to_spa = pd.read_csv('/content/drive/MyDrive/522Project/gpt4/Mariko_gpt.csv', encoding='ISO-8859-1')
gpt4_eng_to_spa = pd.read_csv('/content/drive/MyDrive/522Project/gpt4/Unfit_gpt.csv', encoding='ISO-8859-1')
gpt5_eng_to_spa = pd.read_csv('/content/drive/MyDrive/522Project/gpt4/Wax_gpt.csv', encoding='ISO-8859-1')
gpt6_eng_to_spa = pd.read_csv('/content/drive/MyDrive/522Project/gpt4/Alyssa_gpt.csv', encoding='ISO-8859-1')
gpt7_eng_to_spa = pd.read_csv('/content/drive/MyDrive/522Project/gpt4/Carrie_gpt.csv', encoding='ISO-8859-1')
gpt8_eng_to_spa = pd.read_csv('/content/drive/MyDrive/522Project/gpt4/Awkward_gpt.csv' , encoding='ISO-8859-1')


gpt_eng_to_spa = pd.concat([gpt1_eng_to_spa, gpt2_eng_to_spa, gpt3_eng_to_spa, gpt4_eng_to_spa, gpt5_eng_to_spa, gpt6_eng_to_spa, gpt7_eng_to_spa, gpt8_eng_to_spa]).reset_index(drop=True)
gpt_eng_to_spa.head()

,Original,gpt4
0,"1. Hi everybody, it's your old pal Sarah. Over the past few months, there's been a lot of press, a lot of gowns, a lot of things. And I was in a gown.","1. Hola a todos, soy vuestra vieja amiga Sarah. Durante los últimos meses, ha habido mucha prensa, muchos vestidos, muchas cosas. Y yo llevaba un vestido."
1,"2. So ridiculous. And I had to wear Spanx under it, not for shapewear reasons.","2. Tan ridículo. Y tuve que llevar Spanx debajo, no por razones de modelado corporal."
2,"3. Not that there's anything wrong with that. But I had to because of my thyroid, because it was a tight dress.","3. No es que haya algo malo en eso. Pero tenía que hacerlo por mi tiroides, porque era un vestido ajustado."
3,4. And when I can't wear a tight skirt without shorts or something because I get thigh rub and it's real.,4. Y cuando no puedo llevar una falda ajustada sin pantalones cortos o algo así porque me rozan los muslos y es real.
4,"5. It burns like fire. So I was wearing these Spanx, and then I had to pee.",5. Arde como fuego. Así que llevaba estos Spanx y luego tuve que orinar.


In [22]:
predictions_eng5 = get_predictions(gpt_eng_to_spa['Original'])
predictions_df_eng5 = pd.DataFrame(predictions_eng5)

predictions_spa5 = get_predictions(gpt_eng_to_spa['gpt4'])
predictions_df_spa5 = pd.DataFrame(predictions_spa5)

In [23]:
final_data5 = pd.concat([gpt_eng_to_spa, predictions_df_eng5, predictions_df_spa5], axis=1)

# The original columns from the dataset plus the English sentiment columns
original_plus_eng5 = final_data5.columns.tolist()[:2] + [f'eng_{label}' for label in final_data5.columns[2:8]]

# The Spanish sentiment columns with the 'spa_' prefix
spa_with_prefix5 = [f'spa_{label}' for label in final_data5.columns[8:14]]

# Combine the two lists to get the full set of new column names
new_column_names5 = original_plus_eng5 + spa_with_prefix5

# Assign the new column names to the DataFrame
final_data5.columns = new_column_names5

# Save the dataframe with predictions to a new CSV
final_data5.to_csv('/content/drive/MyDrive/522Project/gpt_SentAnalysis_eng_to_spa.csv', index=False)

In [24]:
final_data5.head()

,Original,gpt4,eng_sadness,eng_joy,eng_love,eng_anger,eng_fear,eng_surprise,spa_sadness,spa_joy,spa_love,spa_anger,spa_fear,spa_surprise
0,"1. Hi everybody, it's your old pal Sarah. Over the past few months, there's been a lot of press, a lot of gowns, a lot of things. And I was in a gown.","1. Hola a todos, soy vuestra vieja amiga Sarah. Durante los últimos meses, ha habido mucha prensa, muchos vestidos, muchas cosas. Y yo llevaba un vestido.",0.007062,0.072203,0.001579,0.904068,0.013819,0.001267,0.010322,0.909404,0.006628,0.060647,0.011391,0.001609
1,"2. So ridiculous. And I had to wear Spanx under it, not for shapewear reasons.","2. Tan ridículo. Y tuve que llevar Spanx debajo, no por razones de modelado corporal.",0.061272,0.006225,0.001015,0.770540,0.157143,0.003804,0.026079,0.121941,0.002988,0.818770,0.028719,0.001503
2,"3. Not that there's anything wrong with that. But I had to because of my thyroid, because it was a tight dress.","3. No es que haya algo malo en eso. Pero tenía que hacerlo por mi tiroides, porque era un vestido ajustado.",0.045625,0.007278,0.014571,0.231334,0.699258,0.001935,0.039704,0.443217,0.024511,0.451561,0.037559,0.003449
3,4. And when I can't wear a tight skirt without shorts or something because I get thigh rub and it's real.,4. Y cuando no puedo llevar una falda ajustada sin pantalones cortos o algo así porque me rozan los muslos y es real.,0.003146,0.988669,0.001564,0.003375,0.002894,0.000352,0.026502,0.779634,0.011363,0.150893,0.028866,0.002743
4,"5. It burns like fire. So I was wearing these Spanx, and then I had to pee.",5. Arde como fuego. Así que llevaba estos Spanx y luego tuve que orinar.,0.097952,0.007664,0.001075,0.143146,0.748272,0.001891,0.032896,0.617970,0.020734,0.259105,0.065417,0.003878
